# Bonus - alternative method Research

THIS NOTEBOOK IS NOT A PART OF THE MAIN PROJECT AND IS SIMPLY A DRAFT THAT I THOUGHT WOULD BE RELEVANT TO INCLUDE

This notebook stands apart from the core project, serving as an exploratory draft that delves into an alternative method of analyzing portfolio returns. I am here examining the dynamics of daily adjusted capital—an approach that diverges from the main analysis where the allocated capital remains constant throughout the evaluation period.

In this exploratory, I recalibrate the portfolio's allocated capital daily, factoring in the accrued Profit and Loss (PnL) to offer a different perspective on capital allocation and its impact on strategy performance. It's crucial to note, however, that this method incorporates numerous assumptions about the daily reinjection of returns into the portfolio's capital. As such, the approaches, syntax, and methodologies employed within this notebook are speculative and lack the rigor of our primary analysis.

This draft should not be construed as a definitive part of the project but rather as an ancillary piece of research observation. It originates from the early stages of my investigation, where I navigated through various methodologies in search of the most apt approach to dissect the project's problem statement. Given the speculative nature of the assumptions regarding capital allocation and the preliminary syntax used, this notebook is presented as a 'bonus' insight rather than a fundamental component of our research.

As we traverse through this alternative analysis, it's imperative to view it through the lens of exploratory research—a space where hypotheses are tested, assumptions are made, and methodologies are trialed in the pursuit of deeper understanding.

## 1. Pre-processing

In [1]:
import pandas as pd
import numpy as np

In [2]:
excel_path = '/Users/raffaello/Projet Interview/Interview Project Input.xlsx'

df_pnl = pd.read_excel(excel_path, sheet_name='Interview Project Input')
df_capital = pd.read_excel(excel_path, sheet_name='Allocated Capital')

# Convert column 'Date' in datetime
df_pnl['Date'] = pd.to_datetime(df_pnl['Date'])
df_capital['Date'] = pd.to_datetime(df_capital['Date'])

# Sort the Dates
df_pnl = df_pnl.sort_values(by='Date')
df_capital = df_capital.sort_values(by='Date')

# Create a dataframe of Dates covering the entire range of Dates
all_dates = pd.date_range(start=min(df_pnl['Date'].min(), df_capital['Date'].min()), 
                          end=df_pnl['Date'].max(), freq='D')

# Mix allocated capital and Date
daily_capital = pd.DataFrame(all_dates, columns=['Date']).merge(df_capital, on='Date', how='left')

# Use 'forward fill for missing value in Daily Capital
daily_capital.fillna(method='ffill', inplace=True)

# Mix PnL with Daily Capital
pnl_with_capital = pd.merge(df_pnl, daily_capital, on='Date', how='left')

# Calculate the total daily PnL
pnl_daily_total = pnl_with_capital.groupby('Date')[' PnL '].sum()

# Create a dataframe with the total daily PnL
df_merged = pd.merge(pnl_with_capital, pnl_daily_total, on='Date')

## 2. Setting and initialize the Daily Adjusted Capital

In [3]:
#Initialize the Adjusted Capital with the Initial Allocated Capital
capital_adjusted = daily_capital.set_index('Date')
capital_adjusted['Adjusted Capital'] = capital_adjusted['Capital']


In [4]:
# Ensure the DataFrame is sorted by date
capital_adjusted.sort_index(inplace=True)
pnl_daily_total.sort_index(inplace=True)

# Initialize the adjusted capital on the first day
capital_adjusted.at[capital_adjusted.index[0], 'Adjusted Capital'] = capital_adjusted.at[capital_adjusted.index[0], 'Capital']

# Loop through the capital_adjusted DataFrame starting from the second day
for i in range(1, len(capital_adjusted)):
    current_date = capital_adjusted.index[i]
    prev_date = capital_adjusted.index[i - 1]

    # Check if there's a change in capital allocation
    if capital_adjusted.at[current_date, 'Capital'] != capital_adjusted.at[prev_date, 'Capital']:
        # Reset the Adjusted Capital to the new Capital without applying PnL
        capital_adjusted.at[current_date, 'Adjusted Capital'] = capital_adjusted.at[current_date, 'Capital']
    else:
        # If no change in capital, apply daily PnL adjustments if PnL data exists for the current date
        if current_date in pnl_daily_total.index:
            capital_adjusted.at[current_date, 'Adjusted Capital'] = capital_adjusted.at[prev_date, 'Adjusted Capital'] + pnl_daily_total[current_date]
        else:
            # If no PnL data for the current date, carry forward the Adjusted Capital from the previous day
            capital_adjusted.at[current_date, 'Adjusted Capital'] = capital_adjusted.at[prev_date, 'Adjusted Capital']


In [5]:
# Convert the index into datetime
capital_adjusted.index = pd.to_datetime(capital_adjusted.index)

# Shift the value to the first Capital allocation switch 
limit_date = '2017-10-18'
capital_adjusted.loc[:limit_date, 'Adjusted Capital'] = capital_adjusted.loc[:limit_date, 'Adjusted Capital'].shift(1)

In [6]:
# Checking if the value are correctly adapting to the Capital Adjustment switch
capital_adjusted['2017-10-18':'2017-10-20']

,Capital,Adjusted Capital
Date,,
2017-10-18,250000000.0,2.742578e+08
2017-10-19,300000000.0,3.000000e+08
2017-10-20,300000000.0,2.990825e+08


In [7]:
capital_adjusted['2018-12-20':'2018-12-21']

,Capital,Adjusted Capital
Date,,
2018-12-20,300000000.0,2.939491e+08
2018-12-21,225000000.0,2.250000e+08


## 3. Construction of the combined DataFrame

In [8]:
if capital_adjusted.index.name == 'Date':
    capital_adjusted.reset_index(inplace=True)

# Perform the merge
df_merged_final = pd.merge(df_merged, capital_adjusted[['Date', 'Adjusted Capital']], on='Date', how='left')

# Rename the column
df_merged_final.rename(columns={' PnL _x': 'PnL', ' PnL _y': 'Daily PnL Total'}, inplace=True)

#Calculation of the Daily Return Total based on Daily PnL Total compared to the Adjusted Capital
df_merged_final['Daily Return Total']=df_merged_final['Daily PnL Total']/df_merged_final['Adjusted Capital']

#Calculation of the Strat Return based on their PnL compared to the Adjusted Capital of the day
df_merged_final['Strat Return']=df_merged_final['PnL']/df_merged_final['Adjusted Capital']

In [9]:
pd.set_option('display.float_format', '{:.10f}'.format)


#df_merged_final.head(30)

## 4. Construction of the window and top N parameter

In [10]:
def calculate_and_store_window_data(df, window_size, top_N):
    # Ensure the DataFrame's index is a DatetimeIndex for time-based operations. If not, it convert the 'Date' column to datetime and set it as the index
    if not isinstance(df.index, pd.DatetimeIndex):
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)

    # Filter out weekends by keeping only the rows where the day of the week is less than 5
    df = df[df.index.dayofweek < 5]
    
    # Group by index (Date) and take the first 'Daily Return Total' for each day/assumes each day has at least one entry
    daily_returns = df.groupby(df.index)['Daily Return Total'].first()

    # Initialize the start index for the first window.
    current_start_index = 0
    # Obtain the unique business days from the daily returns index for iterating over
    business_days = daily_returns.index.unique()
    
    # Initialize an empty list to store data about each window
    window_data_list = []

    # Iterate over the business days in steps of the window size until all days are covered
    while current_start_index < len(business_days):
        # Determine the end index of the current window, ensuring it does not exceed the array bounds
        window_end_index = min(current_start_index + window_size - 1, len(business_days) - 1)
        # Extract the start and end dates of the current window
        window_start = business_days[current_start_index]
        window_end = business_days[window_end_index]

        # Slice the DataFram to only include data within the current window
        window_data = df[window_start:window_end]
        # Calculate the cumulative return for each strategy in the window, then get the top N strategies by return
        strategy_returns = window_data.groupby('Strategy')['Strat Return'].apply(
            lambda x: (1 + x).prod() - 1
        ).nlargest(top_N)

        # List of top N strategies based on return
        top_strategies = strategy_returns.index.tolist()
        # Calculate the sum of PnL for the top N strategies and for all strategies within the window
        top_n_pnl = window_data[window_data['Strategy'].isin(top_strategies)]['PnL'].sum()
        total_pnl = window_data['PnL'].sum()
        # Calculate the concentration of return for the top N strategies.
        concentration_return = top_n_pnl / total_pnl if total_pnl != 0 else 0
        # Calculate the total return for the period based on 'Daily Return Total'
        period_daily_returns = daily_returns[window_start:window_end]
        total_period_return = (1 + period_daily_returns).prod() - 1

        # Store the calculated metrics for the current window in a dictionary and append it to the list
        window_data_list.append({
            'Period Start': window_start,
            'Period End': window_end,
            'Period Return': total_period_return,
            'PnL total period': total_pnl,
            f'Top {top_N} Strategies': top_strategies,
            'PnL top N strat': top_n_pnl,
            'Return Concentration top N': concentration_return
        })

        # Move to the next window by incrementing the start index by the window size.
        current_start_index += window_size

    # Convert the list of dictionaries into a DataFrame for easy analysis and return it.
    return pd.DataFrame(window_data_list)


In [11]:
# usage example
df_results = calculate_and_store_window_data(df_merged_final, window_size=20, top_N=5)

In [12]:
df_results.head(20)

,Period Start,Period End,Period Return,PnL total period,Top 5 Strategies,PnL top N strat,Return Concentration top N
0,2017-03-07,2017-04-03,-0.0146209398,-3655234.9600000000,"[428, 429, 350, 407, 389]",1361378.2200000000,-0.3724461587
1,2017-04-04,2017-05-02,0.0193030026,4755193.6500000013,"[249, 487, 506, 590, 448]",2728773.3999999990,0.5738511617
2,2017-05-03,2017-05-31,-0.0239697693,-6018808.0900000222,"[349, 468, 428, 418, 364]",3289376.5499999984,-0.5465162705
3,2017-06-01,2017-06-28,0.0434652493,10652513.3200000059,"[398, 831, 830, 249, 573]",9543673.5599999987,0.8959081555
4,2017-06-29,2017-07-27,0.0290210649,7421663.2499999963,"[468, 573, 553, 830, 514]",6251739.7799999993,0.8423637087
5,2017-07-28,2017-08-24,-0.0114320376,-3008401.5999999931,"[468, 487, 398, 553, 831]",5283186.4399999995,-1.7561440068
6,2017-08-25,2017-09-22,0.0274322113,7136405.4399999809,"[368, 405, 831, 353, 468]",11260050.2499999795,1.5778321936
7,2017-09-25,2017-10-20,0.0194572057,5075953.4499999983,"[558, 420, 1063, 357, 506]",8898386.4499999974,1.7530472920
8,2017-10-23,2017-11-17,-0.0084551414,-2218531.6800000006,"[680, 558, 437, 357, 368]",9922990.3399999961,-4.4727737852
9,2017-11-20,2017-12-18,-0.0284119914,-8180661.3300000066,"[630, 1270, 602, 665, 673]",8407049.8099999987,-1.0276736160


## 5. Observation only for the positive return period

In [13]:
# Filtered on only positive value
df_positive_return=df_results[df_results['Period Return']>0]

In [14]:
#Storing the report
df_positive_return.to_excel("Bonus_Positive_Return_Period.xlsx")

In [15]:
df_positive_return

,Period Start,Period End,Period Return,PnL total period,Top 5 Strategies,PnL top N strat,Return Concentration top N
1,2017-04-04,2017-05-02,0.0193030026,4755193.6500000013,"[249, 487, 506, 590, 448]",2728773.3999999990,0.5738511617
3,2017-06-01,2017-06-28,0.0434652493,10652513.3200000059,"[398, 831, 830, 249, 573]",9543673.5599999987,0.8959081555
4,2017-06-29,2017-07-27,0.0290210649,7421663.2499999963,"[468, 573, 553, 830, 514]",6251739.7799999993,0.8423637087
6,2017-08-25,2017-09-22,0.0274322113,7136405.4399999809,"[368, 405, 831, 353, 468]",11260050.2499999795,1.5778321936
7,2017-09-25,2017-10-20,0.0194572057,5075953.4499999983,"[558, 420, 1063, 357, 506]",8898386.4499999974,1.7530472920
10,2017-12-19,2018-01-18,0.0157121891,4724186.1999999993,"[1212, 754, 602, 1270, 893]",7075626.4699999979,1.4977450444
12,2018-02-16,2018-03-16,0.0035623491,1221306.7199999914,"[754, 556, 1511, 978, 468]",8408109.3499999959,6.8845190257
13,2018-03-19,2018-04-16,0.0188013202,5513522.3499999950,"[556, 1063, 587, 923, 978]",10566218.8699999992,1.9164189785
14,2018-04-17,2018-05-14,0.0241016853,6991096.0499999784,"[1778, 397, 368, 630, 754]",7406143.5899999933,1.0593680214
20,2018-10-04,2018-10-31,0.0128113517,2975777.4999999888,"[673, 417, 1063, 1232, 744]",9661579.3899999931,3.2467411928
